Caixeiro viajante
=================



## Introdução



Até agora, sempre que nós aplicamos os operadores de `cruzamento` ou de `mutação` nós não nos preocupamos se o indivíduo gerado por estes processo era um `indivíduo válido`. Um indivíduo válido é aquele que representa uma solução possível e bem formulada para o problema em questão.

Por exemplo, no problema das caixas binárias, [1, 0, 0, 1] é um indivíduo válido para o caso de termos 4 caixas. Um exemplo de `indivíduo inválido` para este mesmo problema seria [1, 0, 0, a], pois um dos genes está assumindo um valor fora do domínio. Outro exemplo de indivíduo inválido poderia ser [1, 1, 0], pois é um indivíduo com apenas 3 genes, sendo que o esperado eram 4 genes.

Neste experimento nós veremos estratégias para evitar que indivíduos inválidos sejam obtidos quando usamos os operadores de cruzamento e de mutação. No notebook seguinte veremos como aplicar uma penalidade para indivíduos inválidos que forem gerados durante uma busca genética com restrições.



## Objetivo



Encontrar uma solução para o problema do caixeiro viajante. Considere que ele irá visitar 5 cidades, pode iniciar sua viagem por qualquer uma destas cidades e deve retornar à cidade de início. Durante seu trajeto, não pode visitar a mesma cidade duas vezes (única exceção é a cidade inicial).



## Descrição do problema



O problema consiste em descobrir a rota de menor distância entre $n$ pontos no plano cartesiano (ou seja, $n$ pontos com coordenadas $(x,y)$). A rota pode se iniciar em qualquer um dos pontos disponíveis e deve terminar no ponto inicial, visitando todos os demais pontos apenas uma vez. Considere que a rota entre um ponto e outro é a linha reta que liga os dois pontos.


## Importações



In [1]:
import random
from itertools import permutations

from funcoes import cria_cidades

from funcoes import populacao_inicial_cv as cria_populacao_inicial
from funcoes import funcao_objetivo_pop_cv
from funcoes import selecao_torneio_min as funcao_selecao  # esse já temos!
from funcoes import cruzamento_ordenado as funcao_cruzamento
from funcoes import mutacao_de_troca as funcao_mutacao

from funcoes import funcao_objetivo_cv

## Códigos e discussão



In [2]:
### CONSTANTES

# relacionadas à busca
TAMANHO_POP = 50
NUM_GERACOES = 1000
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.05
NUM_COMBATENTES_NO_TORNEIO = 3

# relacionadas ao problema a ser resolvido
NUMERO_DE_CIDADES = 5
CIDADES = cria_cidades(NUMERO_DE_CIDADES)

In [3]:
# Funções locais

def funcao_objetivo_pop(populacao):
    return funcao_objetivo_pop_cv(populacao, CIDADES)

def funcao_objetivo_individuo(individuo):
    return funcao_objetivo_cv(individuo, CIDADES)

In [4]:
# Busca por algoritmo genético

populacao = cria_populacao_inicial(TAMANHO_POP, CIDADES)

melhor_fitness_ja_visto = float("inf")  # é assim que escrevemos infinito em python

for n in range(NUM_GERACOES):
    
    # Seleção
    fitness = funcao_objetivo_pop(populacao)
    populacao = funcao_selecao(populacao, fitness)
    
    # Cruzamento
    pais = populacao[0::2]
    maes = populacao[1::2]
    
    contador = 0
    
    for pai, mae in zip(pais, maes):
        if random.random() <= CHANCE_CRUZAMENTO:
            filho1, filho2 = funcao_cruzamento(pai, mae)
            populacao[contador] = filho1
            populacao[contador + 1] = filho2
        
        contador = contador + 2   
        
    # Mutação
    for n in range(len(populacao)):
        if random.random() <= CHANCE_MUTACAO:
            individuo = populacao[n]
            populacao[n] = funcao_mutacao(individuo)            
            
    # melhor individuo já visto até agora
    fitness = funcao_objetivo_pop(populacao)
    menor_fitness = min(fitness)
    if menor_fitness < melhor_fitness_ja_visto:        
        posicao = fitness.index(menor_fitness)
        melhor_individuo_ja_visto = populacao[posicao]
        melhor_fitness_ja_visto = menor_fitness  

In [5]:
# Busca testando todas as permutações

melhor_fitness_ever = float("inf")

# testando todas as permutações possíveis
for caminho in permutations(list(CIDADES.keys())):
    distancia = funcao_objetivo_individuo(caminho)
    if distancia < melhor_fitness_ever:
        melhor_fitness_ever = distancia
        melhor_resposta_ever = caminho

In [6]:
# Checando os resultados

print()
print("Melhor individuo obtido por algoritmos genéticos:")
print(melhor_individuo_ja_visto, "com distância:", melhor_fitness_ja_visto)

print()
print("Melhor individuo obtido por busca exaustiva:")
print(melhor_resposta_ever, "com distância:", melhor_fitness_ever)


Melhor individuo obtido por algoritmos genéticos:
['Cidade 1', 'Cidade 0', 'Cidade 2', 'Cidade 4', 'Cidade 3'] com distância: 2.6661206871476018

Melhor individuo obtido por busca exaustiva:
('Cidade 0', 'Cidade 1', 'Cidade 3', 'Cidade 4', 'Cidade 2') com distância: 2.6661206871476018


## Conclusão



Nesse experimento, utilizamos os algoritmos genéticos para resolver o problema do caixeiro viajante. É importante entender que o problema do caixeiro viajante é um problema bem complexo e que até hoje não existe um algoritmo que consiga resolvê-lo determinísticamente ou que prove que não dá para resolvê-lo com algoritmos. Então é essencial entender que os algoritmos genéticos são capazes de dar uma resposta boa para o problema dentro das condições dadas a ele (como o número de gerações) mas é importante entender como os algoritmos genéticos fazem isso. Criar um algoritmo que teste todas as possibilidades de combinações e distâncias para o caixeiro é inviável computacionalmente, dessa forma, o critério de parada estabelecido para resolver esse problema tem que ser outro, o número de gerações é uma boa alternativa. Com o número de gerações determinados, teremos a menor distância possível que o algoritmo achar nessas condições. Hoje ao longo das funções que criamos, vimos a importância de ter uma estrutura de dados que faça sentido para o problema e simplifique o código. Bom, esse foi o primeiro problema que tivemos que apresenta restrições, o que reflete no indivíduo, na seleção, no cruzamento e na mutação. Tivemos, então, que trocar muitos dos tipos de funções que utilizamos anteriormente. Na mutação, utilizamos a mutação de troca que sorteia dois genes (nesse caso, as cidades) e troca os valores entre esses genes então o primeiro assume o valor do segundo e vice versa. No cruzamento, fizemos o ordenado, que tem dois pontos de corte e conta com um loop para que os genes que serão adicionados aos filhos não sejam repetidos no indivíduo. Essas duas funções, cada uma a sua maneira garantem que materemos as mesmas cidades e nenhuma será repetida nem retirada. Fizemos além dos AG uma busca por permutação que testa todas as possibilidades, ou seja, é um algoritmo determinístico. Usei o resultado da busca por permutação para comparar com o do algoritmos genético, que deram exatamente o mesmo resultado. Concluo, portanto, que os algoritmos genéticos funcionam muito bem para resolver o problema do caixeiro viajante e problemas com restrições, o que torna eles muito aplicáveis em problemas da vida real.  

## Playground



O critério de parada tem que ser o número de gerações. Não tem como testar todas as possibilidade. Estrutura de dados - é importante simplificar a estrutura não precisa carregar tudo, ela só precisa carregar o que faz sentido para você. 
Mutação de troca - sorteia dois genes ao invés de 1 e troca os dados, isso para não repetir dados.
Cruzamento ordenado - dois pontos de corte - não pode ser o mesmo ponto de corte (mas pode ser o incio). No ponto simples, poderia repetir o gene, o que não é permitido nesse problema. Pega os primeiros genes do pai, percorre a mãe e ve se todos os gene da mãe já estão no pai, os que não estão serão acrescentados no final. Os primeiros genes são colocados a partir do primeiro corte e o restante começa desde o início do indivíduo da mãe.

A função objetivo que temos que minimizar é a distância, entre otodos os pontos e entre o final e o inicial.

PRIBLEMA COM CONDIÇÕES, restrições

gene = uma cidade?
inividuo = um cria cidades, ou seja, cria um conjunto de cidades. lista com os nomes das cidades
população =
todos os individuos tem a mesma cidade!

A permutação é determinística